<h2> 1. Import libraries </h2>

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import yfinance as yf

from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import minimize

from Data_Retriever import DataRetriever as DatRet
from Markowitz_PT import MarkowitzPT as MPT
# from BHB_Analyzer import BHBAnalyzer as BHBA

<h2> 2. Define experimental variables </h2>

In [2]:
# Define number of trades
trading_n = 10

In [3]:
# Defining stock pool
ticker_df =  pd.DataFrame()
ticker_df["Petroleum"] = ["EQNR.OL", "AKRBP.OL", ]
ticker_df["Seafood (food)"] = ["ORK.OL", "LSG.OL"]

<h2> 3. Retrieve data </h2>

In [4]:
# Retrieve data from yf API: y-m-d
data = DatRet(ticker_df, "2012-10-01", "2025-02-18")

# In function below, set log=True to check for data availability
data.retrieve_data()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


'--Data retrieved successfully--'

<h2> 4. Frequency trading using MPT </h2>

In [5]:
# Generate benchmark weights thorugh MPT using Sharpe ratio
benchmark = MPT(data.returns, 10,trading_n)
# IMPORTANT: In order to see  the effect of the weights, algo exclude last observation from optimization
benchmark.frequency_optimizing()

--Frequency trading using MPT successfully performed--


<h2> 5. Conduct performance analysis </h2>

In [51]:
ben = np.array([0.3, 0.1, 0.4, 0.2])
exp = np.array([0.4, 0.3, 0.2, 0.1])
ret = np.array([0.023, -0.01, -0.017, 0.0023])

In [48]:
n_sec = 2
n_stocks = 2

def analyzer_time_t(ret:list, we:list, wb:list):


    # Returns and weights on sector level for  benchmark
    rb = np.array([wb[i*n_stocks:(i+1) * n_stocks]@ret[int(i*n_stocks):(i+1)*n_stocks] for i in range(n_sec)])
    wb = np.array([sum(wb[i*n_stocks:(i+1)*n_stocks]) for i in range(n_sec)])
    
    # Returns and weights on sector level for portfolio
    re = np.array([we[i*n_stocks:(i+1)*n_stocks]@ret[int(i*n_stocks):(i+1)*n_stocks] for i in range(n_sec)])
    we = np.array([sum(we[i*n_stocks:(i+1)*n_stocks]) for i in range(n_sec)])

    # Total portfolio and benchmark return
    Re = np.dot(we,re)
    Rb = np.dot(wb,rb)

    # Naked allocation and selection effects
    sel_nkd = (1+we*re) / (1+ we*rb) -1
    all_nkd = (1 + (we-wb)*rb) /  (1+(we-wb)*Rb) -1

    # Q value, in three equations for simplicity
    q_top =  np.log(1+Re) - np.log(1+Rb) - sum(np.log((1+sel_nkd)*(1+all_nkd)))
    q_bottom = sum(np.log2(1+sel_nkd)) + sum(np.log2(1+all_nkd))
    q_tot =  q_top/q_bottom
    
    # Optimized selection and allocation effects, on sector level.
    sel_opt = (1+sel_nkd) * np.e**(q_tot*np.log2(1+sel_nkd))
    all_opt = (1+all_nkd) * np.e**(q_tot*np.log2(1+all_nkd))

    # Returns allocation and slection in a list on sector level
    return [all_opt, sel_opt]

In [53]:
svar = analyzer_time_t(ret, exp,  ben )
svar

[array([1.00220325, 1.00146756]), array([1.00020905, 1.00095242])]

In [9]:
# svar = [[],[],[]]

In [10]:
# # observation X weights
# len(benchmark.frequency_weights)

In [11]:
# return_array =  [pd.Series(stock) for sector in data.returns for stock in sector]

In [12]:
# return_array =  [pd.Series(stock) for sector in data.returns for stock in sector]
# return_df = pd.DataFrame(return_array)
# return_tdf = return_df.T

In [13]:
# return_tdf = return_df.T

In [14]:
# benchmark.frequency_weights[0][0]

In [15]:
# len(data.returns)

In [16]:
# return_tdf[0].tail(10)

In [17]:
# trading_obs = 10

In [18]:
# benchmark.frequency_weights

In [71]:
# class MencheroOGA():
#     """
#     one-liner

#     attributes

#     methods
#     """

#     def __init__(self, returns, benchmark_w, experimental_w):
#         """s
#         one-liner
        
#         Args
#         """
#         self.returns = returns
#         self.benchmark_w =  benchmark_w
#         self.experimental_w = experimental_w

#         self.n_optimizations: int = len(self.benchmark_w)
        
#         self.n_sectors = len(self.returns)
#         self.n_stocks = len(self.benchmark_w[0][0]) / self.n_sectors   # Per sector

#         self.allocation_effects: list = None
#         self.selection_effects: list = None

        

#     def analyzer_time_t(self, ret:list, we:list, wb:list):
#         """
#         one-liner

#         Args

#         Returns
#         """
#         # Returns and weights on sector level for  benchmark
#         rb = np.array([wb[i*n_stocks:(i+1) * n_stocks]@ret[int(i*n_stocks):(i+1)*n_stocks] for i in range(n_sec)])
#         wb = np.array([sum(wb[i*n_stocks:(i+1)*n_stocks]) for i in range(n_sec)])
        
#         # Returns and weights on sector level for portfolio
#         re = np.array([we[i*n_stocks:(i+1)*n_stocks]@ret[int(i*n_stocks):(i+1)*n_stocks] for i in range(n_sec)])
#         we = np.array([sum(we[i*n_stocks:(i+1)*n_stocks]) for i in range(n_sec)])

#         # Total portfolio and benchmark return
#         Re = np.dot(we,re)
#         Rb = np.dot(wb,rb)

#         # Naked allocation and selection effects
#         sel_nkd = (1+we*re) / (1+ we*rb) -1
#         all_nkd = (1 + (we-wb)*rb) /  (1+(we-wb)*Rb) -1

#         # Q value, in three equations for simplicity
#         q_top =  np.log(1+Re) - np.log(1+Rb) - sum(np.log((1+sel_nkd)*(1+all_nkd)))
#         q_bottom = sum(np.log2(1+sel_nkd)) + sum(np.log2(1+all_nkd))
#         q_tot =  q_top/q_bottom
        
#         # Optimized selection and allocation effects, on sector level.
#         sel_opt = (1+sel_nkd) * np.e**(q_tot*np.log2(1+sel_nkd))
#         all_opt = (1+all_nkd) * np.e**(q_tot*np.log2(1+all_nkd))

#         # Returns allocation and slection in a list on sector level
#         return [all_opt, sel_opt]



#     def frequnecy_analyzer(self):
#         """
#         One liner
        
#         Args
        
#         Returns
#         """
#         return_array =  [pd.Series(stock) for sector in self.returns for stock in sector]
#         return_df = pd.DataFrame(return_array)
#         return_tdf = return_df.T

#         relevant_return_list = [return_tdf.iloc[-self.n_optimizations+i] for i in range(self.n_optimizations)]
#         # relevant_benW_list = self.benchmark_w

#         # MOCK DATA SO FAR!!!
#         total_n_stocks = self.n_stocks * self.n_sectors
#         relevant_expW_list = [[np.repeat(1/total_n_stocks, total_n_stocks)] for i in range(self.n_optimizations)]

#         allocation_list = []
#         selection_list = []

#         for time in range(self.n_optimizations):
#             # print(relevant_expW_list[time][0])
#             effects = analyzer_time_t(relevant_return_list[time], self.benchmark_w[time][0], relevant_expW_list[time][0])
#             allocation_list.append(effects[0])
#             selection_list.append(effects[1])
        
#         self.allocation_effects = np.concatenate(allocation_list)
#         self.selection_effects = np.concatenate(selection_list)

#         print("--Frequency analysis performed succesfully--")

In [20]:
benchmark.frequency_weights[9]

[array([2.34822419e-01, 1.19298980e-04, 4.34893492e-01, 3.30164791e-01]),
 0.9982103538737593,
 np.float64(0.008650835847296144)]

In [63]:
anal = MencheroOGA(data.returns, benchmark.frequency_weights, [])
anal.frequnecy_analyzer()

--Frequency analysis performed succesfully--


In [68]:
np.cumprod(anal.allocation_effects)[-1]

np.float64(0.9817131870378915)

In [70]:
anal.selection_effects

array([0.9426491 , 1.11218194, 0.94061982, 1.18255596, 0.9390196 ,
       1.19938829, 0.97193216, 1.2967599 , 0.95283767, 1.25802975,
       0.99914711, 1.33248867, 0.93774804, 1.17227577, 0.93882036,
       1.18219066, 0.93815134, 1.16812928, 0.93588958, 1.16902119])

In [69]:
np.cumprod(anal.selection_effects)

array([0.9426491 , 1.04839731, 0.98614328, 1.16616962, 1.09505613,
       1.3133975 , 1.27653327, 1.65535716, 1.57728666, 1.98427354,
       1.98258117, 2.64176694, 2.47731177, 2.90409256, 2.72642122,
       3.2231497 , 3.02380222, 3.53219192, 3.30574162, 3.86448201])

In [345]:
# bomb = [[2,3],[1,5]]
# np.concatenate(bomb)

In [346]:
# [[np.repeat(0.11,4)] for i in range(10)]